In [85]:
import os
import re
import sqlite3 as sql
import emoji
import datetime
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

plt.style.use('fivethirtyeight')
#%matplotlib inline

In [101]:
if not os.path.exists('data/fivethirtyeight_tweets.csv'):
    with open('data/fivethirtyeight_tweets.csv', 'w'):
        for ii in range(1,12):
            print('Accessing file '+str(ii)+' of 11...')
            if ii == 1:
                ira_tweets_538 = pd.read_csv('https://raw.githubusercontent.com/fivethirtyeight/russian-troll-tweets/master/IRAhandle_tweets_'+str(ii)+'.csv')
            else:
                more = pd.read_csv('https://raw.githubusercontent.com/fivethirtyeight/russian-troll-tweets/master/IRAhandle_tweets_'+str(ii)+'.csv')
                ira_tweets_538 = pd.concat([ira_tweets,more])
        ira_tweets_538.to_csv('data/fivethirtyeight_tweets.csv')
        print('Done.')
        pass
else:
    print('Opening existing data file...')
    ira_tweets_538 = pd.read_csv('data/fivethirtyeight_tweets.csv')
    print('Done.')

Opening existing data file...


/Users/gcdunn/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,11,16,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Done.


In [102]:
ira_tweets_538.head()

,Unnamed: 0,external_author_id,author,content,region,language,publish_date,harvested_date,following,followers,...,account_type,retweet,account_category,new_june_2018,alt_external_id,tweet_id,article_url,tco1_step1,tco2_step1,tco3_step1
0,0,906000000000000000,10_GOP,"""We have a sitting Democrat US Senator on tria...",Unknown,English,10/1/2017 19:58,10/1/2017 19:59,1052,9636,...,Right,0,RightTroll,0,905874659358453760,914580356430536707,http://twitter.com/905874659358453760/statuses...,https://twitter.com/10_gop/status/914580356430...,NaN,NaN
1,1,906000000000000000,10_GOP,Marshawn Lynch arrives to game in anti-Trump s...,Unknown,English,10/1/2017 22:43,10/1/2017 22:43,1054,9637,...,Right,0,RightTroll,0,905874659358453760,914621840496189440,http://twitter.com/905874659358453760/statuses...,https://twitter.com/damienwoody/status/9145685...,NaN,NaN
2,2,906000000000000000,10_GOP,Daughter of fallen Navy Sailor delivers powerf...,Unknown,English,10/1/2017 22:50,10/1/2017 22:51,1054,9637,...,Right,1,RightTroll,0,905874659358453760,914623490375979008,http://twitter.com/905874659358453760/statuses...,https://twitter.com/10_gop/status/913231923715...,NaN,NaN
3,3,906000000000000000,10_GOP,JUST IN: President Trump dedicates Presidents ...,Unknown,English,10/1/2017 23:52,10/1/2017 23:52,1062,9642,...,Right,0,RightTroll,0,905874659358453760,914639143690555392,http://twitter.com/905874659358453760/statuses...,https://twitter.com/10_gop/status/914639143690...,NaN,NaN
4,4,906000000000000000,10_GOP,"19,000 RESPECTING our National Anthem! #StandF...",Unknown,English,10/1/2017 2:13,10/1/2017 2:13,1050,9645,...,Right,1,RightTroll,0,905874659358453760,914312219952861184,http://twitter.com/905874659358453760/statuses...,https://twitter.com/realDonaldTrump/status/914...,NaN,NaN


In [103]:
ira_tweets_538.columns

Index(['Unnamed: 0', 'external_author_id', 'author', 'content', 'region',
       'language', 'publish_date', 'harvested_date', 'following', 'followers',
       'updates', 'post_type', 'account_type', 'retweet', 'account_category',
       'new_june_2018', 'alt_external_id', 'tweet_id', 'article_url',
       'tco1_step1', 'tco2_step1', 'tco3_step1'],
      dtype='object')

In [104]:
ira_tweets_538.content.head()

0    "We have a sitting Democrat US Senator on tria...
1    Marshawn Lynch arrives to game in anti-Trump s...
2    Daughter of fallen Navy Sailor delivers powerf...
3    JUST IN: President Trump dedicates Presidents ...
4    19,000 RESPECTING our National Anthem! #StandF...
Name: content, dtype: object

Drop columns that I will not use

In [105]:
ira_tweets_538 = ira_tweets_538.drop(['region','Unnamed: 0','article_url','tco1_step1','tco2_step1','tco3_step1'], axis=1)

In [106]:
ira_tweets_538.head()

,external_author_id,author,content,language,publish_date,harvested_date,following,followers,updates,post_type,account_type,retweet,account_category,new_june_2018,alt_external_id,tweet_id
0,906000000000000000,10_GOP,"""We have a sitting Democrat US Senator on tria...",English,10/1/2017 19:58,10/1/2017 19:59,1052,9636,253,NaN,Right,0,RightTroll,0,905874659358453760,914580356430536707
1,906000000000000000,10_GOP,Marshawn Lynch arrives to game in anti-Trump s...,English,10/1/2017 22:43,10/1/2017 22:43,1054,9637,254,NaN,Right,0,RightTroll,0,905874659358453760,914621840496189440
2,906000000000000000,10_GOP,Daughter of fallen Navy Sailor delivers powerf...,English,10/1/2017 22:50,10/1/2017 22:51,1054,9637,255,RETWEET,Right,1,RightTroll,0,905874659358453760,914623490375979008
3,906000000000000000,10_GOP,JUST IN: President Trump dedicates Presidents ...,English,10/1/2017 23:52,10/1/2017 23:52,1062,9642,256,NaN,Right,0,RightTroll,0,905874659358453760,914639143690555392
4,906000000000000000,10_GOP,"19,000 RESPECTING our National Anthem! #StandF...",English,10/1/2017 2:13,10/1/2017 2:13,1050,9645,246,RETWEET,Right,1,RightTroll,0,905874659358453760,914312219952861184


Remove website urls

In [107]:
ira_tweets_538['content'] = ira_tweets_538.content.str.replace('((www\.[^\s]+)|(http?://[^\s]+)|(https?://[^\s]+))','')

Add two new columns: list of hashtags in tweet, list of @ mentions

In [108]:
'''
#.*?     -- non-greedy match for words starting with #
(?=\s|$) -- look ahead for the end of the word
'''
ira_tweets_538['hashtags'] = ira_tweets_538.content.str.findall(r'#.*?(?=\s|$)')
ira_tweets_538['mentions'] = ira_tweets_538.content.str.findall(r'@.*?(?=\s|$)')

Select only tweets in English

In [109]:
eng_tweets = ira_tweets_538[ira_tweets_538.language == 'English']

Remove emojis

In [111]:
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U0001F1F2-\U0001F1F4"  # Macau flag
        u"\U0001F1E6-\U0001F1FF"  # flags
        u"\U0001F600-\U0001F64F"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U0001F1F2"
        u"\U0001F1F4"
        u"\U0001F620"
        u"\u200d"
        u"\u2640-\u2642"
        "]+", flags=re.UNICODE)

eng_tweets.content = eng_tweets.content.apply(lambda x: emoji_pattern.sub(r'', str(x)))

/Users/gcdunn/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


Remove @mentions

In [112]:
eng_tweets.content = eng_tweets.content.str.replace('@[^\s]+','')

/Users/gcdunn/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


Remove hashtags

In [123]:
eng_tweets.content = eng_tweets.content.str.replace('#[^\s]+','')

/Users/gcdunn/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


Remove RTs

In [124]:
eng_tweets.content = eng_tweets.content.str.replace('RT','')

/Users/gcdunn/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


Remove punctuation

In [125]:
eng_tweets.content = eng_tweets.content.apply(lambda x: re.sub('[@#&.,:;?!-*<>\_\'\"]', ' ', str(x)))

/Users/gcdunn/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


Finally, remove any extra whitespace

In [126]:
eng_tweets.content = eng_tweets.content.apply(lambda x: re.sub('[\s]+', ' ', x))

/Users/gcdunn/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


Now split tweets into original content and retweets

In [127]:
user_rt = eng_tweets[eng_tweets.retweet==1].groupby('external_author_id').agg({'author': lambda x: x.iloc[0],'content': lambda x: ''.join(x)})
user_original = eng_tweets[eng_tweets.retweet==0].groupby('external_author_id').agg({'author': lambda x: x.iloc[0],'content': lambda x: ''.join(x)})

Save these dataframes to SQLITE databases

In [128]:
if not os.path.isfile('../data/user_original.db'):
    with sql.connect('../data/user_original.db') as conn:
        user_original.to_sql('user_original', conn, index=False)
        
if not os.path.isfile('../data/user_rt.db'):
    with sql.connect('../data/user_rt.db') as conn:
        user_rt.to_sql('user_rt', conn, index=False)